# Производительность реляционной модели на примере работы с Postgres

<p> https://en.wikipedia.org/wiki/PostgreSQL
<p> http://initd.org/psycopg/

<img src="model.png" alt="Модель" width="300"/>

```sql
CREATE TABLE "region" (
    id SERIAL PRIMARY KEY,
    name VARCHAR(64) NOT NULL);
CREATE TABLE "group" (
    id SERIAL PRIMARY KEY,
    name VARCHAR(256) NOT NULL);
CREATE TABLE "crimestatsocial" (
    id SERIAL PRIMARY KEY,
    reg_id INTEGER NOT NULL,
    year INTEGER NOT NULL,
    group_id INTEGER NOT NULL,
    category VARCHAR(256) NOT NULL,
    gender CHAR(8) NOT NULL,
    value INTEGER NOT NULL,
    FOREIGN KEY (reg_id)
        REFERENCES "region" (id)
        ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY (group_id)
        REFERENCES "group" (id)
        ON UPDATE CASCADE ON DELETE CASCADE);
```

In [1]:
import sys
import time

import psycopg2

sys.path.append('../config/')
from config import config

In [2]:
params = config()
conn = psycopg2.connect(**params)
cur = conn.cursor()

In [3]:
commands = (
    """
    SELECT
        region.id, region.name, crimestatsocial.year,
        "group".id, "group".name, crimestatsocial.category,
        crimestatsocial.gender, crimestatsocial.value
    FROM crimestatsocial, region, "group"
    WHERE crimestatsocial.group_id = "group".id
    AND crimestatsocial.reg_id = region.id;""",
    """
    SELECT
        region.id, region.name, crimestatsocial.year,
        "group".id, "group".name, crimestatsocial.category,
        crimestatsocial.gender, crimestatsocial.value
    FROM crimestatsocial, region, "group"
    WHERE crimestatsocial.group_id = "group".id
    AND crimestatsocial.reg_id = region.id
    AND crimestatsocial."year"=2016;""",
    """
    SELECT group_id, COUNT("group_id")
    FROM "crimestatsocial"
    WHERE "year"=2016
    GROUP BY "group_id";""",
    """
    SELECT year, COUNT(*)
    FROM "crimestatsocial"
    GROUP BY year;""",
    """
    SELECT COUNT("id") AS count
    FROM "group";"""
)

## 1. Собираются исходные кортежи

In [4]:
cur.execute(commands[0])
fetchall = cur.fetchall()
print(len(fetchall))
print(fetchall[0])

27384
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины ', 13573)


In [5]:
%timeit cur.execute(commands[0])

302 ms ± 44.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2. Собираются исходные кортежи за 2016 год

In [6]:
cur.execute(commands[1])
fetchall = cur.fetchall()
print(len(fetchall))
print(fetchall[0])

3948
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины ', 13573)


In [7]:
%timeit cur.execute(commands[1])

52.5 ms ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 3. Подсчет исходных кортежей за 2016 год по группам

In [8]:
cur.execute(commands[2])
cur.fetchall()

[(14002, 1128), (14007, 376), (14006, 1692), (14005, 752)]

In [9]:
%timeit cur.execute(commands[2])

15.3 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 4. Подсчет исходных кортежей по годам

In [10]:
cur.execute(commands[3])
cur.fetchall()

[(2011, 3864),
 (2010, 3864),
 (2014, 3990),
 (2013, 3864),
 (2015, 3990),
 (2016, 3948),
 (2012, 3864)]

In [11]:
%timeit cur.execute(commands[3])

33.1 ms ± 5.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 5. Подсчет уникальных групп

In [12]:
def fun():
    cur.execute(commands[4])
    fetchall = cur.fetchall()
    return fetchall[0][0]

fun()

4

In [13]:
%timeit fun()

766 µs ± 77.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
cur.close()
conn.close()